In [ ]:

import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
sales_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.STORE_SALES") \
    .select(
        col("SS_ITEM_SK").alias("item_sk"),
        col("SS_STORE_SK").alias("store_sk"),
        col("SS_QUANTITY").alias("quantity"),
        col("SS_SALES_PRICE").alias("sales_price")
    ) \
    .filter(col("SS_SALES_PRICE").is_not_null()) \
    .limit(10000)  # Limit to 10k rows for faster processing

sales_table.show(5)
df = sales_table.to_pandas()
df.columns = [c.lower() for c in df.columns]  # lowercase column names
X = df[['item_sk', 'store_sk', 'quantity']]
y = df['sales_price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.2f}, R2: {r2:.2f}")
predictions_df = pd.DataFrame({
    'item_sk': X_test['item_sk'].values,
    'store_sk': X_test['store_sk'].values,
    'quantity': X_test['quantity'].values,
    'actual_sales_price': y_test.values,
    'predicted_sales_price': y_pred
})
predictions_snowpark = session.create_dataframe(predictions_df)
session.sql("CREATE DATABASE IF NOT EXISTS MY_DB").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS MY_DB.MY_SCHEMA").collect()
predictions_snowpark.write.mode("overwrite").save_as_table("MY_DB.MY_SCHEMA.SALES_PREDICTIONS")
predictions_snowpark
pred_pd = predictions_snowpark.limit(1000).to_pandas()
plt.figure(figsize=(8,6))
sns.scatterplot(
    x='actual_sales_price',
    y='predicted_sales_price',
    data=pred_pd,
    alpha=0.6
)
plt.plot(
    [pred_pd['actual_sales_price'].min(), pred_pd['actual_sales_price'].max()],
    [pred_pd['actual_sales_price'].min(), pred_pd['actual_sales_price'].max()],
    color='red', linestyle='--'
)
plt.title("Actual vs Predicted Sales Price")
plt.xlabel("Actual Sales Price")
plt.ylabel("Predicted Sales Price")
plt.show()

# Error distribution
pred_pd['error'] = pred_pd['predicted_sales_price'] - pred_pd['actual_sales_price']
plt.figure(figsize=(8,5))
sns.histplot(pred_pd['error'], bins=50, kde=True)
plt.title("Prediction Error Distribution")
plt.xlabel("Error")
plt.ylabel("Count")
plt.show()


In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

sales_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.STORE_SALES") \
    .select(
        col("SS_ITEM_SK").alias("item_sk"),
        col("SS_STORE_SK").alias("store_sk"),
        col("SS_QUANTITY").alias("quantity"),
        col("SS_SALES_PRICE").alias("sales_price")
    ) \
    .filter(col("SS_SALES_PRICE").is_not_null())\
    .limit(10000)

sales_table.show(5)
df = sales_table.to_pandas()
df.columns = [c.lower() for c in df.columns]
# Drop rows with NaN (or alternatively use SimpleImputer)
df = df.dropna(subset=['item_sk', 'store_sk', 'quantity', 'sales_price'])

# Alternatively, to impute missing values:
# imputer = SimpleImputer(strategy='median')
# X_imputed = imputer.fit_transform(df[['item_sk', 'store_sk', 'quantity']])
X = df[['item_sk', 'store_sk', 'quantity']]
y = df['sales_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=1),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "LinearRegression": LinearRegression(),
    "HistGradientBoosting": HistGradientBoostingRegressor(max_iter=100, random_state=42)  # handles NaN directly
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'model': model, 'y_pred': y_pred, 'mse': mse, 'r2': r2}
    print(f"{name} -> MSE: {mse:.2f}, R2: {r2:.2f}")
predictions_df = pd.DataFrame({
    'item_sk': X_test['item_sk'].values,
    'store_sk': X_test['store_sk'].values,
    'quantity': X_test['quantity'].values,
    'actual_sales_price': y_test.values,
    'predicted_rf': results['RandomForest']['y_pred'],
    'predicted_gb': results['GradientBoosting']['y_pred'],
    'predicted_lr': results['LinearRegression']['y_pred'],
    'predicted_hgb': results['HistGradientBoosting']['y_pred']
})

# Convert Pandas DF to Snowpark DF
predictions_snowpark = session.create_dataframe(predictions_df)
session.sql("CREATE DATABASE IF NOT EXISTS MY_DB").collect()
session.sql("CREATE SCHEMA IF NOT EXISTS MY_DB.MY_SCHEMA").collect()
predictions_snowpark.write.mode("overwrite").save_as_table("MY_DB.MY_SCHEMA.SALES_PREDICTIONS")
predictions_snowpark
pred_pd = predictions_snowpark.limit(1000).to_pandas()

# Scatter plot: Actual vs Predicted
plt.figure(figsize=(10,6))
sns.scatterplot(x='actual_sales_price', y='predicted_rf', data=pred_pd, label='RandomForest', alpha=0.6)
sns.scatterplot(x='actual_sales_price', y='predicted_gb', data=pred_pd, label='GradientBoosting', alpha=0.6)
sns.scatterplot(x='actual_sales_price', y='predicted_lr', data=pred_pd, label='LinearRegression', alpha=0.6)
sns.scatterplot(x='actual_sales_price', y='predicted_hgb', data=pred_pd, label='HistGradientBoosting', alpha=0.6)
plt.plot([pred_pd['actual_sales_price'].min(), pred_pd['actual_sales_price'].max()],
         [pred_pd['actual_sales_price'].min(), pred_pd['actual_sales_price'].max()],
         color='red', linestyle='--', label='Perfect Prediction')
plt.title("Actual vs Predicted Sales Price for Multiple Models")
plt.xlabel("Actual Sales Price")
plt.ylabel("Predicted Sales Price")
plt.legend()
plt.show()

# Error distribution for each model
pred_pd['error_rf'] = pred_pd['predicted_rf'] - pred_pd['actual_sales_price']
pred_pd['error_gb'] = pred_pd['predicted_gb'] - pred_pd['actual_sales_price']
pred_pd['error_lr'] = pred_pd['predicted_lr'] - pred_pd['actual_sales_price']
pred_pd['error_hgb'] = pred_pd['predicted_hgb'] - pred_pd['actual_sales_price']

plt.figure(figsize=(10,5))
sns.histplot(pred_pd['error_rf'], bins=50, color='blue', kde=True, label='RandomForest', alpha=0.5)
sns.histplot(pred_pd['error_gb'], bins=50, color='green', kde=True, label='GradientBoosting', alpha=0.5)
sns.histplot(pred_pd['error_lr'], bins=50, color='orange', kde=True, label='LinearRegression', alpha=0.5)
sns.histplot(pred_pd['error_hgb'], bins=50, color='purple', kde=True, label='HistGradientBoosting', alpha=0.5)
plt.title("Prediction Error Distribution for Multiple Models")
plt.xlabel("Error")
plt.ylabel("Count")
plt.legend()
plt.show()
